In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import math
from __future__ import division

In [2]:
df = pd.read_excel('formatted_data.xlsx')

means = df.mean(axis = 1, numeric_only  = True)
#print means
#df = df.replace('?', np.nan)
df = df.replace('?', means)
print df
newdf = df.drop('UserID', axis = 1)
newdf

                      UserID Recipe 1 Recipe 2 Recipe 3 Recipe 4 Recipe 5  \
0   5a07ce6c2968911d482dd0ae        0        0        0        0        0   
1   5a0aff3f2d47904af1b93f72        ?        ?        ?        ?        ?   
2   5a0b05dc2d47904af1b93f73        ?        ?        ?        ?        ?   
3   5a0b06692d47904af1b93f74        ?        ?        ?        ?        ?   
4   5a0b0bdd2d47904af1b93f75        ?        ?        ?        ?        ?   
5   5a0b1cc42d47904af1b93f76        ?        ?        ?        ?        ?   
6   5a0b4c492d47904af1b93f77        ?        ?        ?        ?        ?   
7   5a0b4d812d47904af1b93f78        ?        ?        ?        ?        ?   
8   5a0b4ea92d47904af1b93f79        ?        ?        ?        ?        ?   
9   5a0b68452d47904af1b93f7a        ?        ?        ?        ?        ?   
10  5a0b6f282d47904af1b93f7b        ?        ?        ?        ?        ?   
11  5a0b70bc2d47904af1b93f7c        ?        ?        0        ?        ?   

,Recipe 1,Recipe 2,Recipe 3,Recipe 4,Recipe 5,Recipe 6,Recipe 7,Recipe 8,Recipe 9,Recipe 10,...,Recipe 367,Recipe 368,Recipe 369,Recipe 370,Recipe 371,Recipe 372,Recipe 373,Recipe 374,Recipe 375,Recipe 376
0,0,0,0,0,0,0,0,0,0,0,...,?,?,?,?,?,?,?,?,?,?
1,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
2,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
3,?,?,?,?,?,?,?,?,1,?,...,?,?,?,?,?,?,?,?,?,?
4,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
5,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
6,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
7,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
8,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
9,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?


In [3]:
training_set = newdf[0:31]

test_set = newdf[31:37]


Adding all of the ratings corresponding to each recipe in a list of lists to make computation easier.

In [4]:
def makeArrays(dataframe): 
    recipeRatings = []
    #print recipeRatings
    for col in dataframe:
        lists = []
        for val in dataframe[col]:
            lists.append(val) 
        recipeRatings.append(lists)
    print recipeRatings
    return recipeRatings

In [5]:
def averages(array):
    i = 0
    for ratings in array:
        sums = 0
        count = 1
        for rating in ratings:

            if(not math.isnan(rating)):
                sums  = sums + rating
                count = count +1
            average = float(sums/count)
            #print average
            array[i] = [average if math.isnan(x) else float(x) for x in array[i]]
        i = i+1

    #returns a numpy array
    return np.array(array)

# K-means

In [6]:
training_data = averages(makeArrays(training_set))
kmeans = KMeans(n_clusters=4, random_state=0).fit(training_data)

[[0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?'], [0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', 1, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?'], [0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', 0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?'], [0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', 1, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?'], [0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?'], [0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u

TypeError: a float is required

In [ ]:
print kmeans.labels_

In [7]:
recipeRatings = []
#print recipeRatings
for col in training_set:
    lists = []
    for val in training_set[col]:
        lists.append(val) 
    recipeRatings.append(lists)
print recipeRatings

[[0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?'], [0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', 1, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?'], [0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', 0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?'], [0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', 1, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?'], [0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?'], [0, u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u'?', u

In [8]:
i = 0
for ratings in recipeRatings:
    sums = 0
    count = 1
    for rating in ratings:

        if(not math.isnan(rating)):
            sums  = sums + rating
            count = count +1
        average = float(sums/count)
            #print average
        recipeRatings[i] = [average if math.isnan(x) else float(x) for x in recipeRatings[i]]
    i = i+1

    #returns a numpy array
print np.array(recipeRatings)

TypeError: a float is required